# Применение логистической регрессии для решения задачи классификации

# Описание набора данных

Данный набор содержит информацию о пассажирах корабля «Титаник», затонувшего в ночь на 15-е апреля 1912 года. Некоторое количество пассажиров спаслось, чему способствовало множество различных факторов, включая их пол, возраст, на какой палубе находилась их кабина, социальный статус, и т.д.

Набор данных состоит из различных признаков, описывающих информацию о пассажирах. Каждая строка таблицы — отдельный пассажир, вся информация о нем содержится в его строке.

Описание датасета:
- **Survived** (целевой признак): выжил ли пассажир (0 = Нет, 1 = Да);
- **Pclass**: класс каюты (1 = 1st, 2 = 2nd, 3 = 3rd);
- **Sex**: пол (female или male)
- **Age**: возраст в годах
- **SibSp**: количество братьев, сестёр (в том числе сводных) и супругов на борту
- **Parch**: количество детей и родителей на борту
- **Ticket**: номер билета
- **Fare**: стоимость проезда пассажира
- **Cabin**: номер каюты
- **Embarked**: порт посадки пассажира (C = Cherbourg, Q = Queenstown, S = Southampton).

Необходимо решить задачу классификации и научиться предсказывать целовой признак **Survived** — выживет ли пассажир — по соответсвующим ему нецелевым признакам.


# Подготовка и первичный анализ данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

Установите количество пропущенных значений поля <code>age</code>:

In [ ]:
df = pd.read_csv("titanic_train.csv")

In [ ]:
df['age'].isnull().sum()

199

Определите долю выживших

In [ ]:
df.shape

(981, 12)

In [ ]:
df['survived'].value_counts()

0    606
1    375
Name: survived, dtype: int64

In [ ]:
375/(375+606)

0.382262996941896

Определите долю пропущенных значений в рамках каждого признака и избавьтесь от тех признаков, где доля пропусков больше трети. Также удалите колонку <code>ticket</code>, вряд ли эта информация будет полезной.

In [ ]:
df = df.drop(columns = ['ticket'])
df

,pclass,survived,name,sex,age,sibsp,parch,fare,cabin,embarked,home.dest
0,3,1,"Smyth, Miss. Julia",female,NaN,0,0,7.7333,NaN,Q,NaN
1,3,1,"Glynn, Miss. Mary Agatha",female,NaN,0,0,7.7500,NaN,Q,"Co Clare, Ireland Washington, DC"
2,3,1,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,0,0,7.2292,NaN,C,NaN
3,3,0,"Vovk, Mr. Janko",male,22.0,0,0,7.8958,NaN,S,NaN
4,3,0,"de Pelsmaeker, Mr. Alfons",male,16.0,0,0,9.5000,NaN,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...
976,3,1,"Howard, Miss. May Elizabeth",female,NaN,0,0,8.0500,NaN,S,NaN
977,3,1,"Dorking, Mr. Edward Arthur",male,19.0,0,0,8.0500,NaN,S,"England Oglesby, IL"
978,2,0,"Gillespie, Mr. William Henry",male,34.0,0,0,13.0000,NaN,S,"Vancouver, BC"
979,3,0,"Barton, Mr. David John",male,22.0,0,0,8.0500,NaN,S,"England New York, NY"


In [ ]:
missing_values = df.isnull().mean()

# Определите, какие признаки имеют долю пропущенных значений больше трети
features_to_drop = missing_values[missing_values > 1/3].index

# Удаляем выбранные признаки и колонку 'Ticket'
titanic_data = df.drop(features_to_drop, axis=1)

In [ ]:
df = titanic_data
df

,pclass,survived,name,sex,age,sibsp,parch,fare,embarked
0,3,1,"Smyth, Miss. Julia",female,NaN,0,0,7.7333,Q
1,3,1,"Glynn, Miss. Mary Agatha",female,NaN,0,0,7.7500,Q
2,3,1,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,0,0,7.2292,C
3,3,0,"Vovk, Mr. Janko",male,22.0,0,0,7.8958,S
4,3,0,"de Pelsmaeker, Mr. Alfons",male,16.0,0,0,9.5000,S
...,...,...,...,...,...,...,...,...,...
976,3,1,"Howard, Miss. May Elizabeth",female,NaN,0,0,8.0500,S
977,3,1,"Dorking, Mr. Edward Arthur",male,19.0,0,0,8.0500,S
978,2,0,"Gillespie, Mr. William Henry",male,34.0,0,0,13.0000,S
979,3,0,"Barton, Mr. David John",male,22.0,0,0,8.0500,S


Из описания набора данных видно, что колонки <code>sibsp</code> и <code>parch</code> по сути отвечают за размер семьи. Замените эти две колонки на колонку <code>fam_size</code>, значения которой будут вычисляться как сумма соответствующих значений в колонках <code>sibsp</code> и <code>parch</code>.

In [ ]:
# Создаем новую колонку 'Fam_size', сложив 'SibSp' и 'Parch'
df['fam_size'] = df['sibsp'] + df['parch']

# Удаляем колонки 'SibSp' и 'Parch'
df = df.drop(['sibsp', 'parch'], axis=1)

# Результаты
df.head() # Вывод первых строк датасета для проверки

,pclass,survived,name,sex,age,fare,embarked,fam_size
0,3,1,"Smyth, Miss. Julia",female,NaN,7.7333,Q,0
1,3,1,"Glynn, Miss. Mary Agatha",female,NaN,7.7500,Q,0
2,3,1,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,7.2292,C,0
3,3,0,"Vovk, Mr. Janko",male,22.0,7.8958,S,0
4,3,0,"de Pelsmaeker, Mr. Alfons",male,16.0,9.5000,S,0


In [ ]:
mean_Fam_size = titanic_data['Fam_size'].mean()
print(f"Выборочное среднее fnlwgt: {mean_Fam_size:.3f}")

Выборочное среднее fnlwgt: 0.866


Полученный датасет будем называть **исходным** (выброшенные на этом этапе признаки не нужно возвращать ни на одном из последующих этапов выполнения задания). Вычислите полученное число предикторов.

In [ ]:
titanic_data

,pclass,survived,name,sex,age,fare,embarked,Fam_size
0,3,1,"Smyth, Miss. Julia",female,NaN,7.7333,Q,0
1,3,1,"Glynn, Miss. Mary Agatha",female,NaN,7.7500,Q,0
2,3,1,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,7.2292,C,0
3,3,0,"Vovk, Mr. Janko",male,22.0,7.8958,S,0
4,3,0,"de Pelsmaeker, Mr. Alfons",male,16.0,9.5000,S,0
...,...,...,...,...,...,...,...,...
976,3,1,"Howard, Miss. May Elizabeth",female,NaN,8.0500,S,0
977,3,1,"Dorking, Mr. Edward Arthur",male,19.0,8.0500,S,0
978,2,0,"Gillespie, Mr. William Henry",male,34.0,13.0000,S,0
979,3,0,"Barton, Mr. David John",male,22.0,8.0500,S,0


На основе имеющейся статистики оцените вероятность выжить если пассажир -- представитель определенной категории (указана в вашем индивидуальном задании)

In [ ]:
# Фильтруем датасет для пассажиров мужского пола и класса 3
male_pclass3 = titanic_data[(titanic_data['sex'] == 'male') & (titanic_data['pclass'] == 3)]

# Вычисляем долю выживших в этой категории
survival_rate = male_pclass3['survived'].mean()

# Выводим результат
print("Вероятность выживания для мужчин класса 3:", survival_rate)

Вероятность выживания для мужчин класса 3: 0.16442048517520216


Постройте гистограммы выжиших и невыживших по возрастам.

In [ ]:
# < ENTER YOUR CODE HERE >

# Модель на основе числовых признаков

## Удаление строк с пропусками

В качестве базовой модели имеет смысл построить модель с минимальными трудозатратами.

Из исходного датасета удалите все категориальные признаки, а также строки, содержащие пропуски.

При помощи <code>train_test_split()</code> разбейте набор данных на обучающую и тестовую выборки<b> с параметрами, указанными в вашем задании</b>. Используйте стратификацию по колонке <code>survived</code>.

Обучите модель <code>LogisticRegression()</code> <b>с параметрами, указанными в вашем задании</b>, на тренировочных данных, оцените на тестовых.

Вычислите <code>f1_score</code> модели на тестовых данных (рекомендуем использовать <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html">соответствующую функцию</a> с параметрами по умолчанию.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Создайте копию исходного датасета
df = titanic_data.copy()

# Удалите категориальные признаки
categorical_features = df.select_dtypes(include=['object']).columns
df = df.drop(categorical_features, axis=1)

# Удалите строки с пропусками
df = df.dropna()

# Разделите данные на признаки (X) и целевую переменную (y)
X = df.drop('survived', axis=1)
y = df['survived']

# Разбейте данные на обучающую и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Обучите модель логистической регрессии
model = LogisticRegression(random_state=21, max_iter=1000)
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Вычислите f1_score модели на тестовых данных
f1 = f1_score(y_test, y_pred)

# Выведите результат
print("F1-счет модели на тестовых данных:", f1)

F1-счет модели на тестовых данных: 0.5660377358490566


## Заполнение пропусков средним

Качество полученной модели оставляет желать лучшего. Имеет смысл попробовать заполнить пропуски. Из исходного набора данных удалите категориальные признаки, а пропуски заполните средним по столбцу. Далее аналогично: разделение, обучение, оценка.

Вычислите <code>f1_score</code> модели на тестовых данных

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.impute import SimpleImputer

# Создайте копию исходного датасета
df = titanic_data.copy()

categorical_features = df.select_dtypes(include=['object']).columns
df = df.drop(categorical_features, axis=1)

# Используйте SimpleImputer для заполнения пропусков средними значениями
imputer = SimpleImputer(strategy='mean')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)



# Разделите данные на признаки (X) и целевую переменную (y)
X = df.drop('survived', axis=1)
y = df['survived']

# Разбейте данные на обучающую и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Обучите модель логистической регрессии
model = LogisticRegression(random_state=21, max_iter=1000)
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Вычислите f1_score модели на тестовых данных
f1 = f1_score(y_test, y_pred)

# Выведите результат
print("F1-счет модели на тестовых данных:", f1)

F1-счет модели на тестовых данных: 0.4761904761904762


## Заполнение пропусков на основе обращения


Очевидно, что заполнение пропусков в колонке <code>age</code> средним — не самая разумная идея. Возможно, стоит выполнить эту процедуру как-то более интеллектуально, например, с оглядкой на обращение к человеку.

Можно заметить, что исходный датасет в столбце <code>name</code> содержит информацию о социальном статусе пассажира, а именно присутсвуют обращения <code>Mr., Mrs., Dr.</code> и т.д. На основе этой информации можно попробовать сделать предположение о возрасте пассажира.

Верните в рассмотрение колонку <code>name</code>. Создайте отдельную колонку <code>honorific</code> и поместите туда значения обращений.

Вычислите число уникальных обращений.

In [ ]:
import re
titanic_data = pd.read_csv("titanic_train.csv")
# Извлеките обращения из колонки 'Name' с помощью регулярного выражения
titanic_data['honorific'] = titanic_data['name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

# Подсчитайте число уникальных обращений
unique_honorifics = titanic_data['honorific'].nunique()

# Выведите число уникальных обращений
print("Число уникальных обращений:", unique_honorifics)

Число уникальных обращений: 14


Скорее всего имеет смысл сократить число обращений, добавив малочисленные группы к более многочисленным, так как принципиальной разницы между, например, <code>Don</code> и <code>Mr</code>, видимо, нет. Отметим, что <code>Master</code> — это прежнее обращение к ребенку, с этим обращением будем работать отдельно.




Выполните следующие замены:

<code>Mr</code> $\leftarrow$ <code>['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt']</code>

<code>Mrs</code> $\leftarrow$ <code> ['Dona', 'Countess']</code>

<code>Miss</code> $\leftarrow$ <code> ['Mlle', 'Ms']</code>

In [ ]:
# Создайте словарь для выполнения замен
honorific_mapping = {
    'Mr': ['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt'],
    'Mrs': ['Dona', 'Countess'],
    'Miss': ['Mlle', 'Ms']
}

# Функция для выполнения замен
def replace_honorific(honorific):
    for new_honorific, old_honorifics in honorific_mapping.items():
        if honorific in old_honorifics:
            return new_honorific
    return honorific

# Примените функцию для замены обращений
titanic_data['honorific'] = titanic_data['honorific'].apply(replace_honorific)

# Подсчитайте число уникальных обращений после замен
unique_honorifics = titanic_data['honorific'].nunique()

# Выведите число уникальных обращений после замен
print("Число уникальных обращений после замен:", unique_honorifics)

Число уникальных обращений после замен: 4


Вычислите долю строк со значением <code>Master</code> относительно числа всех мужчин

In [ ]:
# Замена обращений согласно заданным правилам
honorific_mapping = {
    'Mr': ['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt'],
    'Mrs': ['Dona', 'Countess'],
    'Miss': ['Mlle', 'Ms']
}

# Применяем замены
for new_honorific, old_honorifics in honorific_mapping.items():
    titanic_data['honorific'].replace(old_honorifics, new_honorific, inplace=True)

# Вычисляем долю строк с обращением 'Master' относительно числа всех мужчин
total_male_passengers = len(titanic_data[(titanic_data['sex'] == 'male')])
master_count = len(titanic_data[(titanic_data['honorific'] == 'Master') & (titanic_data['sex'] == 'male')])
master_ratio = master_count / total_male_passengers

# Выводим результат
print("Доля строк с обращением 'Master' относительно числа всех мужчин:", master_ratio)

Доля строк с обращением 'Master' относительно числа всех мужчин: 0.07232704402515723


Вычислите средний возраст категории, указанной в вашем индивидуальном задании

In [ ]:
# Выберите строки, соответствующие вашей категории (например, 'Mr')
category = 'Mr'
category_age_mean = titanic_data[titanic_data['honorific'] == category]['age'].mean()

# Выведите средний возраст в данной категории
print("Средний возраст в категории", category, ":", category_age_mean)

Средний возраст в категории Mr : 32.90043763676149


Заполните пропуски в колонке <code>age</code> средним, соответствующим среднему категории <code>honorific</code>.

Избавьтесь от нечисловых признаков. Далее аналогично: разделение, обучение, оценка.

Вычислите <code>f1_score</code> модели на тестовых данных

In [ ]:
# Создайте копию исходного датасета
df = titanic_data.copy()

# Заполните пропуски в колонке 'Age' средним значением внутри каждой категории 'Honorific'
df['age'] = df.groupby('honorific')['age'].transform(lambda x: x.fillna(x.mean()))

# Удалите нечисловые признаки и колонку 'Name'
categorical_features = df.select_dtypes(include=['object']).columns
df = df.drop(categorical_features, axis=1)


# Разделите данные на признаки (X) и целевую переменную (y)
X = df.drop('survived', axis=1)
y = df['survived']

# Разбейте данные на обучающую и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Обучите модель логистической регрессии
model = LogisticRegression(random_state=21, max_iter=1000)
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Вычислите f1_score модели на тестовых данных
f1 = f1_score(y_test, y_pred)

# Выведите результат
print("F1-счет модели на тестовых данных после заполнения пропусков и удаления нечисловых признаков:", f1)

F1-счет модели на тестовых данных после заполнения пропусков и удаления нечисловых признаков: 0.5116279069767442


# Модель с использование категориальных признаков

В исходном наборе данных заполните пропуски колонки <code>age</code> значениями на основе обращений (как в предыдущем пункте).

Не используйте признаки <code>name</code> и <code>honorific</code>. Они свою функцию выполнили.

Произведите <code>one-hot</code> кодировние нечисловых признаков, например, с помощью <code>pd.get_dummies(drop_first=True)</code>. Далее по знакомой схеме: разделение, обучение, оценка.

Вычислите <code>f1_score</code> модели на тестовых данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re

In [ ]:
df = pd.read_csv("titanic_train.csv")
df = df.drop(columns = ['ticket'])
missing_values = df.isnull().mean()

# Определите, какие признаки имеют долю пропущенных значений больше трети
features_to_drop = missing_values[missing_values > 1/3].index

# Удаляем выбранные признаки и колонку 'Ticket'
df = df.drop(features_to_drop, axis=1)
# Создаем новую колонку 'Fam_size', сложив 'SibSp' и 'Parch'
df['fam_size'] = df['sibsp'] + df['parch']

# Удаляем колонки 'SibSp' и 'Parch'
df = df.drop(['sibsp', 'parch'], axis=1)

# Результаты
df.head() # Вывод первых строк датасета для проверки

,pclass,survived,name,sex,age,fare,embarked,fam_size
0,3,1,"Smyth, Miss. Julia",female,NaN,7.7333,Q,0
1,3,1,"Glynn, Miss. Mary Agatha",female,NaN,7.7500,Q,0
2,3,1,"Whabee, Mrs. George Joseph (Shawneene Abi-Saab)",female,38.0,7.2292,C,0
3,3,0,"Vovk, Mr. Janko",male,22.0,7.8958,S,0
4,3,0,"de Pelsmaeker, Mr. Alfons",male,16.0,9.5000,S,0


Обучение

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Загрузите исходный датасет
titanic_data = df

titanic_data['honorific'] = titanic_data['name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

# Заполните пропуски в колонке 'Age' средним значением внутри каждой категории 'Honorific'
honorific_mapping = {
    'Mr': ['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt'],
    'Mrs': ['Dona', 'Countess'],
    'Miss': ['Mlle', 'Ms']
}

for new_honorific, old_honorifics in honorific_mapping.items():
    mask = titanic_data['honorific'].isin(old_honorifics)
    titanic_data.loc[mask, 'age'] = titanic_data[mask].groupby('honorific')['age'].transform('mean')

# Выполните one-hot кодирование нечисловых признаков
titanic_data = pd.get_dummies(titanic_data, columns=['sex', 'embarked'], drop_first=True)

# Удалите столбцы 'Name' и 'honorific', так как они уже выполнили свою функцию
titanic_data = titanic_data.drop(['name', 'honorific'], axis=1)
titanic_data = titanic_data.dropna()

# Разделите данные на признаки (X) и целевую переменную (y)
X = titanic_data.drop('survived', axis=1)
y = titanic_data['survived']

# Разбейте данные на обучающую и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Обучите модель логистической регрессии
model = LogisticRegression(random_state=21, max_iter=1000)
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Вычислите f1_score модели на тестовых данных
f1 = f1_score(y_test, y_pred)

# Выведите результат
print(round(f1, 4))

0.7


In [ ]:
# true one
titanic_data = pd.read_csv("titanic_train.csv")

titanic_data = titanic_data.drop(columns = ['ticket'])
missing_values = titanic_data.isnull().mean()

# Определите, какие признаки имеют долю пропущенных значений больше трети
features_to_drop = missing_values[missing_values > 1/3].index

# Удаляем выбранные признаки и колонку 'Ticket'
titanic_data = titanic_data.drop(features_to_drop, axis=1)
# Создаем новую колонку 'Fam_size', сложив 'SibSp' и 'Parch'
titanic_data['fam_size'] = titanic_data['sibsp'] + titanic_data['parch']

# Удаляем колонки 'SibSp' и 'Parch'
titanic_data = titanic_data.drop(['sibsp', 'parch'], axis=1)


titanic_data['honorific'] = titanic_data['name'].apply(lambda x: re.search(' ([A-Za-z]+)\.', x).group(1))

honorific_mapping = {
    'Mr': ['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt'],
    'Mrs': ['Dona', 'Countess'],
    'Miss': ['Mlle', 'Ms']
}

# Функция для выполнения замен
def replace_honorific(honorific):
    for new_honorific, old_honorifics in honorific_mapping.items():
        if honorific in old_honorifics:
            return new_honorific
    return honorific

# Примените функцию для замены обращений
titanic_data['honorific'] = titanic_data['honorific'].apply(replace_honorific)

honorific_mapping = {
    'Mr': ['Rev', 'Col', 'Dr', 'Major', 'Don', 'Capt'],
    'Mrs': ['Dona', 'Countess'],
    'Miss': ['Mlle', 'Ms']
}

# Применяем замены
for new_honorific, old_honorifics in honorific_mapping.items():
    titanic_data['honorific'].replace(old_honorifics, new_honorific, inplace=True)

# Вычисляем долю строк с обращением 'Master' относительно числа всех мужчин
total_male_passengers = len(titanic_data[(titanic_data['sex'] == 'male')])
master_count = len(titanic_data[(titanic_data['honorific'] == 'Master') & (titanic_data['sex'] == 'male')])
master_ratio = master_count / total_male_passengers
category = 'Mr'
category_age_mean = titanic_data[titanic_data['honorific'] == category]['age'].mean()

df = titanic_data.copy()

# Заполните пропуски в колонке 'Age' средним значением внутри каждой категории 'Honorific'
df['age'] = df.groupby('honorific')['age'].transform(lambda x: x.fillna(x.mean()))

# Удалите нечисловые признаки и колонку 'Name'
categorical_features = df.select_dtypes(include=['object']).columns

In [ ]:
df = df.drop(['name', 'honorific'], axis=1)
df = pd.get_dummies(df, columns=['sex', 'embarked'], drop_first=True)


# Разделите данные на признаки (X) и целевую переменную (y)
X = df.drop('survived', axis=1)
y = df['survived']

# Разбейте данные на обучающую и тестовую выборки с учетом стратификации
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

# Обучите модель логистической регрессии
model = LogisticRegression(random_state=21, max_iter=1000)
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Вычислите f1_score модели на тестовых данных
f1 = f1_score(y_test, y_pred)

# Выведите результат
print("F1-счет модели на тестовых данных после заполнения пропусков и удаления нечисловых признаков:", print(round(f1, 3)))

0.735
F1-счет модели на тестовых данных после заполнения пропусков и удаления нечисловых признаков: None


---

# 2

In [ ]:
df_train = pd.read_csv("titanic_train.csv")
df_train = df_train.drop(['name', 'home.dest'], axis=1)

X_test = pd.read_csv("titanic_reserved.csv")
X_test = X_test.drop(['name', 'home.dest'], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_features = df_train.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])

categorical_features = X_test.select_dtypes(include=['object'])

# Initialize a LabelEncoder
label_encoder = LabelEncoder()

# Encode each categorical column
for column in categorical_features.columns:
    X_test[column] = label_encoder.fit_transform(X_test[column])

In [ ]:
imputer = SimpleImputer(strategy='mean')
df_train = pd.DataFrame(imputer.fit_transform(df_train), columns=df_train.columns)

imputer = SimpleImputer(strategy='mean')
X_test = pd.DataFrame(imputer.fit_transform(X_test), columns=X_test.columns)

In [ ]:
X_train = df_train.drop(columns=['survived'])
y_train = df_train['survived']

In [ ]:
# Обучите модель логистической регрессии
model = LogisticRegression(random_state=21, max_iter=1000)
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)
predictions_list = y_pred.tolist()
print(predictions_list)

[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0,

In [ ]:
len(predictions_list)

328

In [ ]:
rounded_predictions = [round(pred) for pred in predictions_list]

# Вывод округленных предсказаний
print(rounded_predictions)

[0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0]
